## 3D SIM reconstruction template

These are the paths where the code resides.  Python needs this so it can load the various utility and helper functions used in this notebook. 

This is a template for running 3D SIM reconstruction based on the Janelia Python and c SIM code written by David Hoffman and Lin Shao.

We have cleaned up the code a bit and added some additional paremeters that may be useful in some cases. 

## 1.  Define code paths
Currently we hard code these and they need to be modified to run on different machines.  In the future we may move to a more intelligent approach like always having code exist beside the notebooks and using relative imports.  

(right now paths are hard-coded for the Janelia machine, and BNs machine where much of the testing has been ongoing)

In [1]:
%pylab inline

import tifffile as tif
import os
import glob
%load_ext autoreload
%autoreload 
import shutil

# NOTE: BN the below code is from the legacy notebooks.  I am leaving it here for now, as it may be needed on some machines. 
if 'C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub' in sys.path:
    sys.path.remove('C:\\Users\\Cryo SIM-PALM\\Documents\\GitHub')
else:
    pass

computer = 'bnort'

import sys

if computer == 'default':
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks')
    sys.path.insert(1, 'Y:\Cryo_data2\Data Processing Notebooks\Scripts')
elif computer == 'bnort':
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts\Scripts')
    sys.path.insert(1, r'C:\Users\bnort\work\Janelia\code\\simrecon\scripts')
else:
    pass

import dphutils 
from simrecon_utils import simrecon, split_process_recombine

# import dask
import dask
from dask.diagnostics import ProgressBar

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Setup home directory and OTF directory:

In this cell we define two paths. 

1.  The ```home``` path which defines where the data is.  The home path will be searched recursively.  All ```.mrc``` files that do not have the string ```proc``` in them will be added to a list of files to be processed. 

2.  The ```otf_path``` directory has the OTFs.  For a file to be processed it must have the wavelength in the file name, and an OTF with the corresponding wavelength must exist. 



#### Additonal legacy notes (BN: these notes were in the original notebook I got) I'm leaving them in for now in case we need them for troubleshhoting

There was an old note "OTF folder should be placed inside Data processing notebooks"

(BN I don't think this has to be the case, because there are many notebook that define the full OTF path)

OTF Folder directory should be e.g.:   Y:\Cryo_data2\Data Processing Notebooks\SIM PSFs OTFs

Raw data from V-SIM data acquisition should be placed inside a dated folder here:   'Y:\Cryo_data2\ORCA_data\3D SIM Cells' 

with data file directory structure e.g.: 'Y:\Cryo_data2\ORCA_data\3D SIM Cells\20240322\488 nm 5 phases 0.81 NA Linear SIM_cam1_0.mrc'

In [2]:
if computer == 'default': 
    home = r'Y:\Cryo_data2\488nm comparison TILED VS Non-Tiled at different base kwarg settings'
    otf_path = r'Y:\Seyforth\Data For Brian\Cryo-SIM Scope #2 Data (James System)\PSFs (best PSFs and examples of bad ones)\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
elif computer == 'bnort':
    #home = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\488nm comparison Brian'
    #home = r'D:\Janelia\Data 2024-10-02\560cm cell 4 _20240627_124604'
    home = r'D:\Janelia\Data 2024-10-10'
    home = r'D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702'
    
    #otf_path = r'D:\Janelia\Data 2024-06-06\Wiener, gammaApo and SupressR parameter testing\OTF\BEAD 2 - NON-AR 1.2W 25ms retake_20240503_170242 BEST PSF!!\computed_OTF_folder'
    otf_path = r'D:\Janelia\Data 2024-06-03\PSF-OTF used (Davids set of 4 wavelengths)\201909_19-20_best'
    otf_path = r'C:\Users\bnort\work\Janelia\ims\OTF_folder'
    otf_path = r'C:\Users\bnort\work\Janelia\ims\OTFs_mixed'
    otf_path = r'D:\Janelia\Data 2024-09-23\Brians OTF'
OTFpath = os.path.join(otf_path,"*{}*.mrc")
OTFs = {wl : [path for path in glob.iglob(OTFpath.format(wl))] for wl in (560, 532, 488, 642)}
OTFs

{560: ['D:\\Janelia\\Data 2024-09-23\\Brians OTF\\560 201909_19-20_best.mrc'],
 532: ['D:\\Janelia\\Data 2024-09-23\\Brians OTF\\532 OTF Bead 3_20190920_154920.mrc'],
 488: ['D:\\Janelia\\Data 2024-09-23\\Brians OTF\\488 nmLinOTF0_mask.mrc'],
 642: ['D:\\Janelia\\Data 2024-09-23\\Brians OTF\\642 20240611_125236_best.mrc']}

## Set default params

Here we set the default params that will be used if none of them are overwritten.

Note that before calling the non-tiled and/or tiled processing code a small subset of parameters will be overwritten to values optimized for non-tiled/tiled processing

In [14]:
# David Solecki's suggestion: wiener=0.007 gammaApo=0.7 and suppressR=15
# original values by D.Hoffman: wiener=0.001 gammaApo=0.1 and suppressR=1.5

base_kwargs = dict(
                    nphases=5,
                    ndirs=3,
                    angle0= 1.29,
                    negDangle=True,              # James made False to try experiment
                    na= 0.85,
                    nimm= 1.0,
                    zoomfact= 2.0, 
                    background= 100.0,           # james experiment was 100.0
                    wiener= 0.007,
                    fastSIM=True,
                    otfRA= True,
                    dampenOrder0=True,
                    k0searchall=True,
                    equalizez=True,
                    preciseapo=True,
                    gammaApo=0.7,
                    suppressR=15.0,
                    nthreads = 1 
                )

def return_wl_otfs(path):
    if "488 Exc 532 Em" in path:
        wl = 532
    elif "488 Exc 642 Em" in path:
        wl = 642
    elif "532 Exc 561 Em" in path:
        wl = 561
    elif "560 nm" in path:
        wl = 560
    elif "488 nm" in path:
        wl = 488
    elif "532 nm" in path:
        wl = 532
    elif "642 nm" in path:
        wl = 642

    else:
        raise RuntimeError("no matching filename wavelength found, fix directory or filename or code")
    return wl, OTFs[wl]

## Tweak params and set advanced params

We tweak the commonly used params (```gammaApo```, ```supressR``` and ```wiener```) and then set some advanced params. 

* ```nofilteroverlaps```: If false we do not mix the overlapping regions of each direction and order.  The theoretically optimal processing weights the frequency components of each band and order by the magnitude of the OTF at that component.  Since bands and orders overlap there can be multiple frequency components at each location and the final component is a OTF weighted mix.  If ```nofilteroverlaps``` is false components will be added together without weighting.  In some cases we have found this results in higher contrast and less striping.  While this result may seem counterintuitive the mixing approach may be sensitive to errors in the OTF (use with care). 
* ```forcemodamp```: This parameter forces the modulation amplitude to be a specific value.  In some cases we have found that using ```[1.0 0.1]``` combined with the ```nofilteroverlaps``` option results in better contrast and less striping. 
* ```forceotfamp``` this option is similar to ```forcemodamp``` but it controls how heavilly the order 1 and order 2 OTF is weighted. 

In [4]:

gammaApo = 0.3
suppressR = 10 
wiener = 0.001 
    
base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener))   # default Full frame Recon. parameters    

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener) 

nofilteroverlaps =False 
forcemodamp = False
forceotfamp = False

if nofilteroverlaps==True:
    user_text += '_nofilteroverlaps'
    base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps))   # default Full frame Recon. parameters    

if forcemodamp:
    o1 = 1.0
    o2 = 0.1
    user_text += '_forcemodamp_'+str(o1)+'_'+str(o2)
    forcemodamp = [o1, o2]
    base_kwargs.update(dict(forcemodamp=forcemodamp))   # default Full frame Recon. parameters    

if forceotfamp:
    o1 = 1
    o2 = 10
    user_text += '_forceotfamp_'+str(o1)+'_'+str(o2)
    forceotfamp = [o1, o2]
    base_kwargs.update(dict(otfamp=forceotfamp))



## remove old processed data

This is commented out, but I assume we comment back in if we want to erase the previous run

In [5]:
'''python
# clear processed
for path in glob.glob(home + "/*/*proc *.mrc"):
    os.remove(path)
for path in glob.glob(home + "/*/*proc *.txt"):
    os.remove(path)
'''

'python\n# clear processed\nfor path in glob.glob(home + "/*/*proc *.mrc"):\n    os.remove(path)\nfor path in glob.glob(home + "/*/*proc *.txt"):\n    os.remove(path)\n'

# Define ```process``` function that uses DASK and simrecon package

This function is annotated as a dask ```delayed``` function.  Which means it will not be called right away but put in a queue to call using dask. 

In [6]:
@dask.delayed
def process(sim_kwargs, output_file_name):
    sim_output = simrecon(**sim_kwargs)
    with open(output_file_name.replace(".mrc", ".txt"), "w") as myfile:
        myfile.write(str(sim_kwargs))
        myfile.write("\n".join(sim_output))
    return output_file_name

# Check for processed data

The cell below checks for raw data which has already been processed using full frame SIM reconstruction algorithm

In [7]:
done_already = set(glob.iglob(home + "/**/*proc*.mrc", recursive = True))
done_already;
done_already = set()     # do this if want to re-process
print('Data processed already: \n')

for i in done_already:
    print(i + '\n')

Data processed already: 



## Set up data to be processed

This cell loops through the ```mrc``` files, looks for an OTF that matches the wavelength in the file name (via ```return_wl_otfs```), makes sure ```proc``` is not in the name, then adds the ```dask.delayed``` ```process``` function to the ```to_process``` list.  The processing will occur in the next cell. 

In [8]:
to_process = []

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    for otf in otfs:
        if "proc" not in raw:
            print('Data to process ' +str(raw))
            # Test with actually measured OTF
            sim_kwargs = dict(                                                                                                            
                input_file= raw,
                otf_file= otf,
                ls= (wl/1000)/2/0.81,)
            
            sim_kwargs.update(base_kwargs)
            OTF_filename = os.path.split(otf)[1].split('.')[0]
            print('\nOTF filename: ' +str(OTF_filename))
                
            #create processed file output name
            sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc", '_proc' + OTF_filename + '_' 
                                                                         + user_text + ".mrc")
                
            if sim_kwargs["output_file"] not in done_already:
                print("\n\nSIM recon Data to save:" + '\n' + str(sim_kwargs["output_file"]) + '\n')
                to_process.append(process(sim_kwargs, sim_kwargs["output_file"]))

Data to process D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc

OTF filename: 560 201909_19-20_best


SIM recon Data to save:
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 201909_19-20_best_gApo_0.3_supR_10_w_0.001.mrc



# Cell below runs all files for full frame reconstruction and progress using DASK

This cell will start and run the processing.  We use dask to compute our processing queue that was set up in the cell above.

In [9]:
with ProgressBar():
    print(to_process)
    out_names = done_already.update(*dask.compute(to_process))

[Delayed('process-3069a60d-82e5-45f3-aac1-8107d4a7d1b2')]
[########################################] | 100% Completed |  2min 28.4s


# Check for data allready processed with tiling

Cell below checks for data already processed using the tiledreconstruction method, cell below that creates list of data to be set to done already, i.e. already processed


In [10]:
for path in glob.glob(home + "/*/*_tile64_pad32*"):
    done_already.add(path.replace("_tile64_pad32", "") + "__split__")

[path for path in done_already if "_tile64_pad32" in path]

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    print(raw)

D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 201909_19-20_best_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 OTF Bead 3_20190920_154442_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_560 OTF Bead 3_20190920_154442_gApo_0.3_supR_0.5_w_0.001_tile16_pad8.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_560 OTF Bead 3_20190920_154442_gAp

In [11]:
print(home)
print(glob.glob(home + "/**/*SIM_*.mrc"))

D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702
[]


# Run tiled reconstruction

Note:  Optimal settings are slightly different for tiled reconstruction and adjusted here.  

```tile_size``` and ```tile_overlap``` can also be adjusted in this cell.

Set ```filter_tiles``` to true will use the ```tile_limits``` dictionary to constrain the line spacings, angles, and amplitudes.  Line spacing is the inverse of the frequency magnitude vector. 

In [15]:
#%%time

gammaApo = 0.5
suppressR = 1.0
wiener = 0.001
done_already = set()
base_kwargs.update(dict(gammaApo=gammaApo, suppressR=suppressR, wiener=wiener)) # default tiling Recon. parameters

user_text = 'gApo_'+str(gammaApo)+'_supR_'+str(suppressR)+'_w_'+str(wiener)

if nofilteroverlaps==True:
    user_text += '_nofilteroverlaps'
    base_kwargs.update(dict(nofilteroverlaps=nofilteroverlaps))   # default Full frame Recon. parameters    

if forcemodamp:
    o1 = 1.0
    o2 = 0.1
    user_text += '_forcemodamp_'+str(o1)+'_'+str(o2)
    forcemodamp = [o1, o2]
    base_kwargs.update(dict(forcemodamp=forcemodamp))   # default Full frame Recon. parameters    

if forceotfamp:
    o1 = 1
    o2 = 10
    user_text += '_forceotfamp_'+str(o1)+'_'+str(o2)
    forceotfamp = [o1, o2]
    base_kwargs.update(dict(otfamp=forceotfamp))


tile_size = 16
tile_overlap = 8

filter_tiles = False
         
if filter_tiles:
    user_text=user_text+'_filter_tiles'

    tile_limits = {}
   
    tile_limits['spacing_min'] = 0.31
    tile_limits['spacing_max'] = 0.33
    tile_limits['spacing_default']=0.315
    tile_limits['angle_min'] = 1.0
    tile_limits['angle_max'] = 2.0
    tile_limits['angle_default'] = 1.36
    tile_limits['amp1_min'] = 0.9
    tile_limits['amp1_max'] = 1.1
    tile_limits['amp1_default'] = 1.0
    tile_limits['amp2_min'] = 0.9
    tile_limits['amp2_max'] = 1.1
    tile_limits['amp2_default'] = 1.0
else:
    tile_limits = None

for raw in glob.iglob(home + "/**/*SIM*.mrc", recursive = True):
    wl, otfs = return_wl_otfs(raw)
    print(raw)
    
    for otf in otfs:
        if "proc" not in raw:                  #checks for if data was already processed
            if "_tile64_pad32" not in raw:       #checks for if data was already processed using tiling method
                print("\nFile to be processed: " + str(raw))
                # Test with actually measured OTF
                sim_kwargs = dict(
                    input_file= raw,
                    otf_file= otf,
                    ls= (wl/1000)/2/0.81
                    #ls = .315
                )
                sim_kwargs.update(base_kwargs)
                
                OTF_filename = os.path.split(otf)[1].split('.')[0]
                
                #create processed file output name
                out_name = sim_kwargs["output_file"] = sim_kwargs["input_file"].replace(".mrc",'_proc_' + OTF_filename + '_' +
                                                                          user_text + ".mrc")
                
                # perform reconstruction and output recon image file and text file
              
                out_name += "__split__"
                
                try:
                    if out_name not in done_already:
                        #print(sim_kwargs["input_file"])
                        sim_output = split_process_recombine(sim_kwargs["input_file"], tile_size, tile_overlap, sim_kwargs, tile_limits=tile_limits)
                        # HAD TO EDIT simrecon_utils.py lines 1488 and 1490 and comment out mrc.close() as it was failing.
                        with open(sim_output[0].replace(".mrc", ".txt"), "w") as myfile:
                            myfile.write(str(sim_kwargs))
                            myfile.write("\n" + "-" * 80 + "\n")
                            myfile.write("\n".join(sim_output[1]))

                    done_already.add(out_name)
                    for path in glob.glob(home + "/*/*_tile64_pad32*"):
                        done_already.add(path.replace("_tile64_pad32", "") + "__split__")
                except:
                    print('Error processing ' + str(raw))
                    continue

D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc

File to be processed: D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1.mrc


  0%|          | 0/64 [00:00<?, ?it/s]

Splitting and saving data:   0%|          | 0/4096 [00:00<?, ?it/s]

[########################################] | 100% Completed | 57.8s


Reading back processed:   0%|          | 0/4096 [00:00<?, ?it/s]

Recombining:   0%|          | 0/4096 [00:00<?, ?it/s]

MRC Raw data file is still open
MRC temp data file is still open
MRC Raw data file is now closed
MRC temp data file is now closed
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 201909_19-20_best_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc560 OTF Bead 3_20190920_154442_gApo_0.3_supR_10_w_0.001.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_560 201909_19-20_best_gApo_0.5_supR_1.0_w_0.001_tile16_pad8.mrc
D:\Janelia\gold_standard_set\position 3 nice cells 561nm 4.74um stack_20240530_153702\560 nm 655 40 filter 5 phases 0.81 NA React_All Linear SIM_cam1_1_proc_560 OTF Bead 3_20190920_154442_gApo_0.3_supR_0.5_w_0.001_tile16_pad8.

###### from simrecon_utils import process_txt_output, plot_params

In [ ]:
dst = "Tiled Parameters/"
try:
    os.mkdir(dst)
except FileExistsError:
    pass
for raw in glob.iglob(home + "/*/*SIM*tile*.txt"):
    with open(raw) as f:
        fig, axs = plot_params(process_txt_output("".join(f.readlines())))
        fig.savefig(dst + os.path.abspath(raw).split(os.path.sep)[-2] + ".png", dpi=300, bbox_inches="tight")

In [ ]:
print("done!")

# Renaming 3D SIM files for DropBox

In [ ]:
for wl in ("488 nm", "532 nm", "560 nm", "642 nm"):
    curdir = os.path.split(os.path.abspath(os.curdir))[1]
    glob_str = "F:/ORCA_Data/{}/3D SIM Cells/*{}*/*{}.mrc".format(curdir, "", "{}*SIM*proc*".format(wl))
    for path in sorted(glob.glob(glob_str)):
        print(path)
        path_head, path_tail = os.path.split(path)
        path_head_new, path_tail_new = os.path.split(path_head)
        file_root = path_tail_new.split("_")[0]+' '+path_tail_new.split("_")[1]

        OTF = path_tail.split('_OTF')[-1]
        proc = OTF.split('_proc')[-1]
        if proc == '.mrc':
            OTF = proc = OTF.split('_proc')[0] + '.mrc'
        new_path = os.path.join(path_head_new, file_root + ' wl'+wl.split(' ')[0]+' proc_with_OTF' + OTF)
        print(new_path)
        print('')
        
        shutil.copyfile(path,new_path)